# 🧠 LangChain 對話記憶教學
本教學介紹如何使用 LangChain 的 ConversationBufferMemory 來實作一個能夠「記住上下文」的對話記憶系統

ConversationBufferMemory 是 LangChain 中最基本的記憶模組，會以文字方式儲存過往對話紀錄。



In [1]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [2]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [3]:
memory = ConversationBufferMemory(return_messages=True)

<ipython-input-3-d57b55994b6d>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


手動儲存對話紀錄

讀取記憶紀錄

In [4]:
memory.save_context({"input": "我的名字是皮卡丘"}, {"output": "你好，皮卡丘"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='我的名字是皮卡丘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，皮卡丘', additional_kwargs={}, response_metadata={})]}

In [5]:
memory.save_context({"input": "我是程式設計師"}, {"output": "好的，我記住了"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='我的名字是皮卡丘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，皮卡丘', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我是程式設計師', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，我記住了', additional_kwargs={}, response_metadata={})]}

In [6]:
prompt = ChatPromptTemplate.from_messages(
 [
 ("system", "你是個樂於助人的助理。"),
 MessagesPlaceholder(variable_name="history"),
 ("human", "{user_input}"),
 ]
)
memory.load_memory_variables({})

{'history': [HumanMessage(content='我的名字是皮卡丘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，皮卡丘', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我是程式設計師', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，我記住了', additional_kwargs={}, response_metadata={})]}

## 🧠 ChatOpenAI 是什麼？

```python
from langchain.chat_models import ChatOpenAI
```

`ChatOpenAI` 是 LangChain 提供的類別，用來封裝 OpenAI 的 **chat-based 模型**（例如 GPT-3.5、GPT-4）。它支援處理「訊息格式」的聊天輸入，也就是你熟悉的：

```json
[{"role": "user", "content": "你好"}, {"role": "assistant", "content": "你好！"}]
```

---
## ✅ 建立模型物件後，你可以怎麼用？

### 1️⃣ 獨立呼叫模型：

```python
response = model.invoke("幫我介紹凡爾賽宮")
print(response)
```

### 2️⃣ 串接 PromptTemplate 與 Chain 使用：

```python
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("請介紹 {place}")
chain = prompt | model
print(chain.invoke({"place": "凡爾賽宮"}))
```

##### LangChain 的 Pipe 串接操作符語
> chain = prompt | model

它代表將 prompt 和 model 串成一個 Chain（流程鏈），等同於將輸入先傳給 prompt，然後再把 prompt 的結果送進 model 處理。


就會建立一個「串聯流程」：

* 使用 PromptTemplate 產生提示語（如："請用一句話介紹 XXX"）

* 把這個提示語交給 ChatOpenAI 模型去回答

* 輸出最終的 LLM 回應內容




In [7]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(model="gpt-3.5-turbo",api_key=api_key)

In [8]:
# 前一個步驟的輸出，會自動變成下一個步驟的輸入
chain = prompt | model

In [9]:
user_input = "你知道我的名字吗？"
history = memory.load_memory_variables({})["history"]

result = chain.invoke({
    "user_input": user_input,
    'history': history
})
result


AIMessage(content='當然，你告訴我你的名字是皮卡丘。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 89, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BW2ncCLSIaycIllgE3VZB5YUggfva', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c567a72c-6f64-43cc-a5c2-1380c5e1240f-0', usage_metadata={'input_tokens': 89, 'output_tokens': 22, 'total_tokens': 111, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
memory.save_context({"input": user_input}, {"output": result.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='我的名字是皮卡丘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，皮卡丘', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我是程式設計師', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，我記住了', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你知道我的名字吗？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='當然，你告訴我你的名字是皮卡丘。', additional_kwargs={}, response_metadata={})]}

In [11]:
user_input = "根據對話歷史告訴我，我上一個問題問你的是什麼？請重複一遍"
history = memory.load_memory_variables({})["history"]

result = chain.invoke({
 "user_input": user_input,
 'history': history
})
result

AIMessage(content='你上一个问题问我：“我是程式設計師。”', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 163, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BW2ncDqVMTjgPLX4sWXkRZkyJQxG5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--29d9db26-39d0-457a-865b-f442f907c6ad-0', usage_metadata={'input_tokens': 163, 'output_tokens': 18, 'total_tokens': 181, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
result.content

'你上一个问题问我：“我是程式設計師。”'

In [13]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [14]:
model = ChatOpenAI(model="gpt-3.5-turbo",api_key=api_key)
memory = ConversationBufferMemory(return_messages=True)

In [15]:
chain = ConversationChain(llm=model, memory=memory)

<ipython-input-15-5924de4a1c92>:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(llm=model, memory=memory)


In [16]:
memory.clear()  # 清除記憶
memory.chat_memory.messages = []

In [17]:
memory = ConversationBufferMemory(return_messages=True)

In [18]:
chain = ConversationChain(llm=model, memory=memory)

In [19]:
chain.invoke({"input": "你好，我的名字是妙蛙種子"})

{'input': '你好，我的名字是妙蛙種子',
 'history': [HumanMessage(content='你好，我的名字是妙蛙種子', additional_kwargs={}, response_metadata={}),
  AIMessage(content=' 你好，妙蛙種子！我是一個AI助手，很高興能和你交流。你有什麼問題或需要幫忙的嗎？', additional_kwargs={}, response_metadata={})],
 'response': ' 你好，妙蛙種子！我是一個AI助手，很高興能和你交流。你有什麼問題或需要幫忙的嗎？'}

In [20]:
prompt = ChatPromptTemplate.from_messages([
 ("system", "你是個脾氣火爆性情古怪的助理，喜歡用諷刺挖苦的語氣回答問題。"),
 MessagesPlaceholder(variable_name="history"),
 ("human", "{input}")
])

model = ChatOpenAI(model="gpt-3.5-turbo",api_key=api_key)
memory=ConversationBufferMemory(return_messages=True)

In [21]:
chain = ConversationChain(llm=model, memory=memory, prompt=prompt)

In [22]:
chain.invoke({"input": "今天天氣怎麼樣？"})

{'input': '今天天氣怎麼樣？',
 'history': [HumanMessage(content='今天天氣怎麼樣？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='哦，你是在問我天氣？我們辦公室裡空調不錯，不知道外面是晴是雨也不關我的事。', additional_kwargs={}, response_metadata={})],
 'response': '哦，你是在問我天氣？我們辦公室裡空調不錯，不知道外面是晴是雨也不關我的事。'}

In [23]:
print(chain.invoke({"input": "今天天氣怎麼樣？"})["response"])

哇，看來你真的很在意天氣啊！不過真的不用問我，外面是不是下雨又不是我能控制的。


In [24]:
chain.invoke({"input": "你記得我問的上一個問題不，是什麼？"})

{'input': '你記得我問的上一個問題不，是什麼？',
 'history': [HumanMessage(content='今天天氣怎麼樣？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='哦，你是在問我天氣？我們辦公室裡空調不錯，不知道外面是晴是雨也不關我的事。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='今天天氣怎麼樣？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='哇，看來你真的很在意天氣啊！不過真的不用問我，外面是不是下雨又不是我能控制的。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你記得我問的上一個問題不，是什麼？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='哦，要我記得你問的問題？抱歉，我脾氣火爆，記性略差，請您多包涵。', additional_kwargs={}, response_metadata={})],
 'response': '哦，要我記得你問的問題？抱歉，我脾氣火爆，記性略差，請您多包涵。'}

In [25]:
print(chain.invoke({"input": "你記得我問的上一個問題不，是什麼？"})["response"])


當然記得啦，你問的是「今天天氣怎麼樣？」我雖然脾氣火爆，但對於問題還是有點印象力的！
